In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt_kwargs = {'figsize': (10, 4)}
from IPython.display import HTML

In [2]:
roles = {"duelist": {"jett", "phoenix", "reyna", "raze", "yoru", "neon", "iso"},
             "initiator": {"sova", "breach", "skye", "kayo", "fade", "gekko"},
             "controller": {"brimstone", "omen", "viper", "astra", "harbor"},
             "sentinel": {"cypher", "sage", "killjoy", "chamber", "deadlock"}}
allagents = set()
for role, l in roles.items():
    allagents = allagents.union(set(l))

In [3]:
alpha = 5 #Eco rating parameters
beta = -5

In [4]:
#Rating for a round: winning with low credits earns more rating than winning with high credits, losing with more credits earns more negative rating.
def EcoRoundRating(LoadOutA, LoadoutB,  OutcomeA):
    LoadoutB = max(LoadoutB, 100)
    LoadOutA = max(LoadOutA, 100)
    if OutcomeA == 1:
        return alpha*LoadoutB/LoadOutA
    if OutcomeA == 0:
        return beta*LoadOutA/LoadoutB
    
def EcoRoundRating1(LoadOutA, LoadoutB, Eco_type_A, Eco_type_B,  OutcomeA):
    LoadoutB = max(LoadoutB, 100)
    LoadOutA = max(LoadOutA, 100)
    if Eco_type_A == Eco_type_B:
        if OutcomeA == 1:
            return alpha*LoadoutB/LoadOutA
        if OutcomeA == 0:
            return beta*LoadOutA/LoadoutB
    if Eco_type_A != Eco_type_B:
        if OutcomeA == 1:
            return LoadoutB/LoadOutA
        if OutcomeA == 0:
            return LoadOutA/LoadoutB
    

In [5]:
players_stats = pd.read_csv("../data/vct_2022/players_stats/players_stats.csv")
overview = pd.read_csv("../data/vct_2022/matches/overview.csv")
maps_scores = pd.read_csv("../data/vct_2022/matches/maps_scores.csv")
eco_rounds = pd.read_csv("../data/vct_2022/matches/eco_rounds.csv")

In [6]:
maps_scores["Team_A_win"] = maps_scores["Team A Score"] > maps_scores["Team B Score"]
maps_scores["Team_B_win"] = maps_scores["Team B Score"] > maps_scores["Team A Score"]

maps_scores = maps_scores[["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A", "Team A Score", "Team_A_win", "Team B", "Team B Score", "Team_B_win"]]

In [7]:
len(players_stats[players_stats["Rating"].isna()])

135

In [8]:
len(eco_rounds[eco_rounds["Loadout Value"].isna()])

0

In [9]:
players_stats = players_stats[['Tournament', 'Stage', 'Match Type', 'Player', 'Team', 'Agents', 'Rating']]

players_stats["Agents"] = players_stats["Agents"].apply(lambda x: x.replace(" ", ""))

#players_stats

Filling the none values in player rating with the average rating of the player/team.

In [10]:
players_stats = players_stats[players_stats.Agents.isin(allagents)]
grouped1 = players_stats.groupby(["Player", "Team", "Agents"])
players_stats.loc[:,"Rating"] = grouped1["Rating"].transform(lambda x: x.fillna(x.mean()))
grouped2 = players_stats.groupby(["Player", "Team"])
players_stats["Rating"] = grouped2["Rating"].transform(lambda x: x.fillna(x.mean()))
grouped3 = players_stats.groupby(["Team"])
players_stats["Rating"] = grouped3["Rating"].transform(lambda x: x.fillna(x.mean()))
players_stats.Rating.isna().sum()
players_stats = players_stats[~players_stats.Rating.isna()]
players_stats.sort_values(by=["Rating"], ascending=False).head(20)

,Tournament,Stage,Match Type,Player,Team,Agents,Rating
93970,Champions Tour CIS Stage 1: Challengers 2,Open Qualifier,Round of 64,RHEA,next_viigraem,omen,2.69
114469,Champions Tour Europe Stage 1: Challengers 1,Open Qualifier,Round of 64,marteen,Straight Outta Retirement,raze,2.67
101886,Champions Tour Europe Stage 1: Challengers 2,Open Qualifier,Round of 128,Dev0,Human Tripwires,raze,2.55
110691,Champions Tour Europe Stage 1: Challengers 1,Open Qualifier,Round of 256,speedy,Dame Time,raze,2.42
68926,Champions Tour Korea Stage 1: Challengers,Open Qualifier: Round of 42,Group J,Blueming,Veni Vidi Vici,killjoy,2.39
112604,Champions Tour Europe Stage 1: Challengers 1,Open Qualifier,Round of 256,lowel,Team Heretics,brimstone,2.39
110385,Champions Tour Europe Stage 1: Challengers 1,Open Qualifier,Round of 256,ShadoW,Team Vitality,breach,2.38
16818,Champions Tour Korea Stage 2: Challengers,Open Qualifier,Group F,Sylvan,World Game Star,viper,2.34
80856,Champions Tour Indonesia Stage 1: Challengers,Group Stage,Opening (D),Famouz,BOY WITH LOVE,raze,2.33
99083,Champions Tour Europe Stage 1: Challengers 2,Open Qualifier,Round of 256,souzaseeds,F9 Hetic,chamber,2.29


Another file which has player rating

In [11]:
overview.sort_values(by=["Rating"], ascending=False).head()

,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents,Rating,Average Combat Score,...,Deaths,Assists,Kills - Deaths (KD),"Kill, Assist, Trade, Survive %",Average Damage per Round,Headshot %,First Kills,First Deaths,Kills - Deaths (FKD),Side
104150,Champions Tour Philippines Stage 2: Challengers,Group Stage,Week 1,Team Secret vs South Built Esports,Haven,dispenser,Team Secret,killjoy,5.35,1139.0,...,0.0,0.0,5.0,100%,522.0,60%,0.0,0.0,0.0,defend
18370,Champions Tour North America Stage 2: Challengers,Open Qualifier #1,Round of 128,Clanless vs SYR,Icebox,Kehmicals,Clanless,jett,5.30,1085.0,...,0.0,0.0,4.0,100%,645.0,10%,1.0,0.0,1.0,attack
249676,Champions Tour Philippines Stage 1: Challengers,Open Qualifier,Lower Round 1,Alpha Esports Pro vs GrindSky Esports,Haven,ABATMAN,Alpha Esports Pro,killjoy,5.09,1125.0,...,0.0,0.0,9.0,100%,646.0,43%,0.0,0.0,0.0,attack
177032,Champions Tour North America Stage 1: Challengers,Open Qualifier #1,Round of 128,Complexity vs Burger Gaming,Haven,jcrueL,Complexity,jett,4.93,916.0,...,0.0,0.0,3.0,100%,493.0,50%,1.0,0.0,1.0,defend
266950,Champions Tour Japan Stage 1: Challengers Week 1,Open Qualifier,Group E,IGZIST vs SYSTEM e-Sports,Haven,oitaN,IGZIST,chamber,4.92,1275.0,...,0.0,0.0,5.0,100%,693.0,56%,1.0,0.0,1.0,attack


In this file, overview "rating" and "acs:average combat score" look off.  As in 2023, we will use rating data from players_stats file.

In [12]:
ind1 = overview["Side"] == "both"
ind2 = overview["Map"] != "All Maps"
ind = ind1 & ind2

overview = overview[ind]
overview = overview[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Player',
       'Team', 'Agents']]
overview

,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents
0,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Benkai,Paper Rex,fade
3,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,d4v41,Paper Rex,sage
6,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,mindfreak,Paper Rex,astra
9,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,f0rsakeN,Paper Rex,chamber
12,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Jinggg,Paper Rex,raze
...,...,...,...,...,...,...,...,...
379827,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,Zyppan,FunPlus Phoenix,raze
379830,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,ardiis,FunPlus Phoenix,kayo
379833,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,ANGE1,FunPlus Phoenix,viper
379836,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,Shao,FunPlus Phoenix,astra


In [13]:
rating_df = overview.merge(players_stats, on=['Tournament', 'Stage', 'Match Type', 'Player', "Agents", "Team"], how="left")

rating_df

,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents,Rating
0,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Benkai,Paper Rex,fade,0.96
1,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,d4v41,Paper Rex,sage,1.15
2,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,mindfreak,Paper Rex,astra,1.07
3,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,f0rsakeN,Paper Rex,chamber,1.01
4,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,Jinggg,Paper Rex,raze,0.89
...,...,...,...,...,...,...,...,...,...
88512,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,Zyppan,FunPlus Phoenix,raze,NaN
88513,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,ardiis,FunPlus Phoenix,kayo,1.20
88514,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,ANGE1,FunPlus Phoenix,viper,1.10
88515,Champions Tour CIS Stage 1: Challengers 1,Closed Qualifier,Grand Final,Natus Vincere vs FunPlus Phoenix,Split,Shao,FunPlus Phoenix,astra,1.04


In [14]:
# I will just fill NaN with the average of team rating.
grouped4 = rating_df.groupby(["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team"])
rating_df["Rating"] = grouped4["Rating"].transform(lambda x: x.fillna(x.mean()))

# After first step I found there are still missing values.  Fill them with each individual player's average throughout this data.
grouped5 = rating_df.groupby(["Player"])
rating_df["Rating"] = grouped5["Rating"].transform(lambda x: x.fillna(x.mean()))

rating_df["Rating"] = rating_df.groupby(["Team"])["Rating"].transform(lambda x: x.fillna(x.mean()))

rating_df = rating_df[~rating_df.Rating.isna()]
rating_df.Rating.isna().sum()

0

In [15]:
grouped_rating_df = rating_df.groupby(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team'])["Rating"].agg("sum").reset_index()

grouped_rating_df

,Tournament,Stage,Match Type,Match Name,Map,Team,Rating
0,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Ascent,CERBERUS Esports,3.300000
1,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Ascent,Made in Thailand,5.550000
2,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Bind,CERBERUS Esports,3.750000
3,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Bind,Made in Thailand,5.022993
4,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (B),Persija Esports vs Bonkers,Bind,Bonkers,5.437500
...,...,...,...,...,...,...,...
17698,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Global Esports vs M1syl and friends,Breeze,M1syl and friends,3.860000
17699,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Maruti Peek vs God Particles,Ascent,God Particles,6.860000
17700,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Maruti Peek vs God Particles,Ascent,Maruti Peek,3.912500
17701,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Maruti Peek vs God Particles,Breeze,God Particles,6.210000


Following is one of the main data frame. It has team rating based on player rating.

In [16]:
final_rating_df =\
      maps_scores.set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  .join(grouped_rating_df\
                        .rename(columns={"Team":"Team A", "Rating":"Team_A_rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  )\
                  .reset_index()\
                  .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  .join(grouped_rating_df\
                        .rename(columns={"Team":"Team B", "Rating":"Team_B_rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  )\
                  .reset_index()

print(len(final_rating_df))

final_rating_df = final_rating_df[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A', 'Team_A_win', 'Team_B_win', 'Team B', 'Team_A_rating', 'Team_B_rating']]

print(final_rating_df.isna().sum())

final_rating_df = final_rating_df.dropna()

print(len(final_rating_df))
final_rating_df

8884
Tournament        0
Stage             0
Match Type        0
Match Name        0
Map               0
Team A            0
Team_A_win        0
Team_B_win        0
Team B            0
Team_A_rating    83
Team_B_rating    84
dtype: int64
8792


,Tournament,Stage,Match Type,Match Name,Map,Team A,Team_A_win,Team_B_win,Team B,Team_A_rating,Team_B_rating
0,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Ascent,Made in Thailand,True,False,CERBERUS Esports,5.550000,3.300000
1,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Bind,Made in Thailand,True,False,CERBERUS Esports,5.022993,3.750000
2,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (B),Persija Esports vs Bonkers,Bind,Persija Esports,False,True,Bonkers,4.530000,5.437500
3,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (B),Persija Esports vs Bonkers,Fracture,Persija Esports,True,False,Bonkers,4.725000,4.333333
4,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (B),Persija Esports vs Bonkers,Haven,Persija Esports,True,False,Bonkers,4.733333,5.600000
...,...,...,...,...,...,...,...,...,...,...,...
8879,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Round 1,M1syl and friends vs Salt Esports,Haven,M1syl and friends,True,False,Salt Esports,4.960000,4.960000
8880,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Global Esports vs M1syl and friends,Bind,Global Esports,True,False,M1syl and friends,6.320000,3.750000
8881,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Global Esports vs M1syl and friends,Breeze,Global Esports,True,False,M1syl and friends,5.700000,3.860000
8882,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Maruti Peek vs God Particles,Ascent,Maruti Peek,False,True,God Particles,3.912500,6.860000


Next, we will work the economy data

In [17]:
eco_rounds["Outcome"] = eco_rounds["Outcome"].apply(lambda x: 1 if x == "Win" else 0)
eco_rounds["Loadout Value"] = eco_rounds["Loadout Value"].apply(lambda x: int(x.replace(".","").replace("k","00")))
eco_rounds["Remaining Credits"] = eco_rounds["Remaining Credits"].apply(lambda x: int(x.replace(".","").replace("k","00")))

half_eco_rounds = eco_rounds.loc[ (eco_rounds["Round Number"]<13)]
eco_rounds.head(10)

,Tournament,Stage,Match Type,Match Name,Map,Round Number,Team,Loadout Value,Remaining Credits,Type,Outcome
0,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,1,Paper Rex,3700,100,Eco: 0-5k,0
1,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,1,EDward Gaming,4000,200,Eco: 0-5k,1
2,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,2,Paper Rex,1200,9600,Eco: 0-5k,0
3,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,2,EDward Gaming,15600,2400,Semi-buy: 10-20k,1
4,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,3,Paper Rex,21300,700,Full buy: 20k+,0
5,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,3,EDward Gaming,15100,9500,Semi-buy: 10-20k,1
6,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,4,Paper Rex,8600,8400,Semi-eco: 5-10k,0
7,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,4,EDward Gaming,25700,16100,Full buy: 20k+,1
8,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,5,Paper Rex,22200,2200,Full buy: 20k+,0
9,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,5,EDward Gaming,26100,17000,Full buy: 20k+,1


In [18]:
team_ab_2022 = maps_scores[["Tournament", "Stage", "Match Type", "Match Name", "Map", "Team A", "Team B"]]

team_ab_2022

eco_rounds_for_prob = eco_rounds.drop(columns=["Remaining Credits", "Outcome", "Type"])
eco_rounds_for_prob.head()

,Tournament,Stage,Match Type,Match Name,Map,Round Number,Team,Loadout Value
0,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,1,Paper Rex,3700
1,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,1,EDward Gaming,4000
2,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,2,Paper Rex,1200
3,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,2,EDward Gaming,15600
4,Valorant Champions 2022,Group Stage,Opening (A),Paper Rex vs EDward Gaming,Pearl,3,Paper Rex,21300


In [19]:
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team']

for i in range(10):
    team_ab_2022 =\
    pd.merge(
        team_ab_2022.rename(columns={"Team A":"Team"})\
        ,eco_rounds_for_prob[eco_rounds_for_prob["Round Number"] == i+1].drop(columns=["Round Number"])\
        ,on=keys
    )\
    .rename(columns={"Loadout Value":f"R{i+1}_A_lo"})\
    .rename(columns={"Team":"Team A", "Team B":"Team"})\
    .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team'])\
    .join(eco_rounds_for_prob[eco_rounds_for_prob["Round Number"] == i+1].drop(columns=["Round Number"])\
        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team'])\
    )\
    .reset_index()\
    .rename(columns={"Loadout Value":f"R{i+1}_B_lo"})\
    .rename(columns={"Team":"Team B"})\

team_ab_2022

,Tournament,Stage,Match Type,Match Name,Map,Team B,Team A,R1_A_lo,R1_B_lo,R2_A_lo,...,R6_A_lo,R6_B_lo,R7_A_lo,R7_B_lo,R8_A_lo,R8_B_lo,R9_A_lo,R9_B_lo,R10_A_lo,R10_B_lo
0,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Ascent,CERBERUS Esports,Made in Thailand,2900,3800,16400,...,21900,22600,21700,11400,24100,24400,23300,24300,21400,22500
1,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (A),Made in Thailand vs CERBERUS Esports,Bind,CERBERUS Esports,Made in Thailand,4200,3200,17800,...,24900,22600,23300,6300,23300,24000,23300,20800,23300,8200
2,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (B),Persija Esports vs Bonkers,Bind,Bonkers,Persija Esports,3600,4100,14400,...,23200,26600,22600,24700,23200,24300,23200,11500,21200,24600
3,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (B),Persija Esports vs Bonkers,Fracture,Bonkers,Persija Esports,3500,3800,2200,...,25900,22300,22400,22200,6800,22400,22500,23000,22400,21500
4,Champions Tour Asia-Pacific Stage 1: Challenge...,Group Stage,Decider (B),Persija Esports vs Bonkers,Haven,Bonkers,Persija Esports,3800,3700,1300,...,23500,23300,22300,22900,22200,24300,23300,24000,23700,23900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8397366,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Round 1,M1syl and friends vs Salt Esports,Haven,Salt Esports,M1syl and friends,3300,3200,13800,...,23500,21000,5600,22700,23400,24100,23700,22300,23400,22300
8397367,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Global Esports vs M1syl and friends,Bind,M1syl and friends,Global Esports,3900,4200,14200,...,23100,17600,21200,20600,22000,6500,22800,23100,21700,22900
8397368,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Global Esports vs M1syl and friends,Breeze,M1syl and friends,Global Esports,3900,4000,15900,...,23400,19300,23500,19900,24500,10100,24500,22100,24500,9500
8397369,Valorant Conquerors Championship,Wildcard Qualifier,Upper Bracket Semifinals,Maruti Peek vs God Particles,Ascent,God Particles,Maruti Peek,3100,3800,1400,...,21700,24100,9300,22700,20900,23000,6700,21800,23100,24400


In [20]:
# keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team']

# Eco_rounds_ab_2022=\
# pd.merge(
#     team_ab_2022.rename(columns={"Team A":"Team"})\
#     ,eco_rounds\
#     ,on=keys
# )\
# .rename(columns={"Loadout Value":"Loadout_A", "Remaining Credits":"Remaining_credit_A", "Type":"Eco_type_A", "Outcome":"Outcome_A"})\
# .rename(columns={"Team":"Team A", "Team B":"Team"})\
# .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team', 'Round Number'])\
# .join(eco_rounds\
#       .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map','Team', 'Round Number'])\
# )\
# .reset_index()\
# .rename(columns={"Loadout Value":"Loadout_B", "Remaining Credits":"Remaining_credit_B", "Type":"Eco_type_B", "Outcome":"Outcome_B"})\
# .rename(columns={"Team":"Team B"})

# #Eco_rounds_ab_2022.head()

MemoryError: Unable to allocate 48.8 GiB for an array with shape (20, 327333763) and data type int64

In [ ]:
#Eco_rounds_ab_2022[[ "Eco_type_A", "Eco_type_B", "Outcome_A", "Round Number"]].groupby([ "Eco_type_A", "Eco_type_B", "Outcome_A"]).count()

In [ ]:
#Eco_rounds_ab_2022["Eco_Round_Rating_A"]=Eco_rounds_ab_2022[["Loadout_A", "Loadout_B", "Outcome_A"]]\
#    .apply(lambda row: EcoRoundRating(row["Loadout_A"], row["Loadout_B"], row["Outcome_A"]), axis=1)

# Eco_rounds_ab_2022["Eco_Round_Rating_B"]=Eco_rounds_ab_2022[["Loadout_A", "Loadout_B", "Outcome_B"]]\
#     .apply(lambda row: EcoRoundRating(row["Loadout_B"], row["Loadout_A"], row["Outcome_B"]), axis=1)


# Eco_rounds_ab_2022.head(10)

# # RoundRating2
# Eco_rounds_ab_2022["Eco_Round_Rating_A"]=Eco_rounds_ab_2022[["Loadout_A", "Loadout_B", "Outcome_A", "Eco_type_A", "Eco_type_B"]]\
#     .apply(lambda row: EcoRoundRating1(row["Loadout_A"],  row["Loadout_B"],row["Eco_type_A"], row["Eco_type_B"], row["Outcome_A"]), axis=1)

# Eco_rounds_ab_2022["Eco_Round_Rating_B"]=Eco_rounds_ab_2022[["Loadout_A", "Loadout_B", "Outcome_B",  "Eco_type_A", "Eco_type_B"]]\
#     .apply(lambda row: EcoRoundRating1(row["Loadout_B"], row["Loadout_A"], row["Eco_type_A"], row["Eco_type_B"], row["Outcome_B"]), axis=1)


# Eco_rounds_ab_2022.head(10)

In [ ]:
#Half_Eco_rounds_ab_2022 = Eco_rounds_ab_2022.loc[Eco_rounds_ab_2022["Round Number"]<13]
#Half_Eco_rounds_ab_2022

Rating for a match is average of the round rating.

In [ ]:
grouped_eco_rating_df_A = Eco_rounds_ab_2022.groupby(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])["Eco_Round_Rating_A"].agg("mean").reset_index()
grouped_eco_rating_df_B = Eco_rounds_ab_2022.groupby(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])["Eco_Round_Rating_B"].agg("mean").reset_index()


In [ ]:
half_grouped_eco_rating_df_A = Half_Eco_rounds_ab_2022.groupby(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])["Eco_Round_Rating_A"].agg("mean").reset_index()
half_grouped_eco_rating_df_B = Half_Eco_rounds_ab_2022.groupby(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])["Eco_Round_Rating_B"].agg("mean").reset_index()

In [ ]:
final_eco_rating_df =\
      maps_scores.set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  .join(grouped_eco_rating_df_A\
                        .rename(columns={"Eco_Round_Rating_A":"Team_A_eco_rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  )\
                  .reset_index()\
                  .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  .join(grouped_eco_rating_df_B\
                        .rename(columns={"Eco_Round_Rating_B":"Team_B_eco_rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  )\
                  .reset_index()

In [ ]:
final_eco_rating_df

In [ ]:
final_rating_df.columns

In [ ]:
final_eco_rating_df = final_eco_rating_df[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A',
       'Team B', 'Team_A_win', 'Team_B_win',
       'Team_A_eco_rating', 'Team_B_eco_rating']]

In [ ]:
final_eco_rating_df.columns

In [ ]:
keys = ['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A',
       'Team B', 'Team_A_win', 'Team_B_win']

final_df=\
pd.merge(
    final_rating_df, final_eco_rating_df,on=keys
)

final_df

In [ ]:
half_final_eco_rating_df =\
      maps_scores.set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  .join(half_grouped_eco_rating_df_A\
                        .rename(columns={"Round_Rating_A":"Team_A_eco_rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A'])\
                  )\
                  .reset_index()\
                  .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  .join(half_grouped_eco_rating_df_B\
                        .rename(columns={"Round_Rating_B":"Team_B_eco_rating"})\
                        .set_index(['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team B'])\
                  )\
                  .reset_index()

In [ ]:
final_eco_rating_df=final_eco_rating_df.dropna()
half_final_eco_rating_df=half_final_eco_rating_df.dropna()

We will train on pre Valorant Champions and test on the championship

In [ ]:
final_df_train = final_df.loc[final_df["Tournament"]!= "Valorant Champions 2022"]
final_df_test  = final_df.loc[final_df["Tournament"]== "Valorant Champions 2022"]
final_df_test = final_df_test[['Tournament', 'Stage', 'Match Type', 'Match Name', 'Map', 'Team A',
       'Team_A_win', 'Team_B_win', 'Team B']]


In [ ]:
#Take ave over team and map

final_rating_df_A_from_before = final_df_train.groupby(['Team A'])["Team_A_rating"].agg("mean").reset_index()
final_rating_df_B_from_before = final_df_train.groupby(['Team B'])["Team_B_rating"].agg("mean").reset_index()
final_eco_rating_df_A_from_before = final_df_train.groupby(['Team A'])["Team_A_eco_rating"].agg("mean").reset_index()
final_eco_rating_df_B_from_before = final_df_train.groupby(['Team B'])["Team_B_eco_rating"].agg("mean").reset_index()
#print(grouped_rating_df_A.head())
print(final_rating_df_A_from_before.head(20))

In [ ]:
final_df_test = final_df_test.merge(final_rating_df_A_from_before, on=["Team A"], how="left")
final_df_test = final_df_test.merge(final_eco_rating_df_A_from_before, on=["Team A"], how="left")
final_df_test

In [ ]:
final_df_test = final_df_test.merge(final_rating_df_B_from_before, on=["Team B"], how="left")
final_df_test = final_df_test.merge(final_eco_rating_df_B_from_before, on=["Team B"], how="left")
#champs_rating_overall_df.isna().sum()
print(len(final_df_test))
final_df_test = final_df_test.dropna()
final_df_train = final_df_train.dropna()
final_df_test


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

Logistic Regression on Team_A_Rating and Team_B_Rating

In [ ]:
lr1 = LogisticRegression()
X1_train = final_df_train[["Team_A_rating", "Team_B_rating"]]
y1_train = final_df_train.Team_A_win.to_numpy().reshape(-1,1)

X1_test = final_df_test[["Team_A_rating", "Team_B_rating"]]
y1_test = final_df_test.Team_A_win.to_numpy().reshape(-1,1)


In [ ]:
print(len(X1_train), len(y1_train), len(X1_test), len(y1_test))

In [ ]:
lr1.fit(X1_train, y1_train)
pred1 = lr1.predict(X1_test)
acc1 = accuracy_score(y1_test, pred1)
print("mean of true values:", np.mean(y1_test))
print("mean of prediction:", np.mean(pred1))
print("Accuracy:", acc1)
lr1.coef_

In [ ]:
cm1 = confusion_matrix(y1_test, pred1, labels=lr1.classes_)
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm1,
                              display_labels=lr1.classes_)
disp1.plot()
plt.show()

Logistic Regression on Team_A_Eco_Rating and Team_B_Eco_Rating

In [ ]:
lr2 = LogisticRegression()
X2_train = final_df_train[["Team_A_eco_rating", "Team_B_eco_rating"]]
y2_train = final_df_train.Team_A_win.to_numpy().reshape(-1,1)

X2_test = final_df_test[["Team_A_eco_rating", "Team_B_eco_rating"]]
y2_test = final_df_test.Team_A_win.to_numpy().reshape(-1,1)

In [ ]:
print(len(X2_train), len(y2_train), len(X2_test), len(y2_test))

In [ ]:
lr2.fit(X2_train, y2_train)
pred2 = lr2.predict(X2_test)
acc2 = accuracy_score(y2_test, pred2)
print("mean of true values:", np.mean(y2_test))
print("mean of prediction:", np.mean(pred2))
print("Accuracy:", acc2)
lr2.coef_

In [ ]:
cm2 = confusion_matrix(y2_test, pred2, labels=lr2.classes_)
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm2,
                              display_labels=lr2.classes_)
disp2.plot()
plt.show()

Logistic Regression on both

In [ ]:
lr3 = LogisticRegression()
X3_train = final_df_train[["Team_A_eco_rating", "Team_B_eco_rating", "Team_A_rating", "Team_B_rating"]]
y3_train = final_df_train.Team_A_win.to_numpy().reshape(-1,1)

X3_test = final_df_test[["Team_A_eco_rating", "Team_B_eco_rating", "Team_A_rating", "Team_B_rating"]]
y3_test = final_df_test.Team_A_win.to_numpy().reshape(-1,1)

In [ ]:
print(len(X3_train), len(y3_train), len(X3_test), len(y3_test))

In [ ]:
lr3.fit(X3_train, y3_train)
pred3 = lr3.predict(X3_test)
acc3 = accuracy_score(y3_test, pred3)
print("mean of true values:", np.mean(y3_test))
print("mean of prediction:", np.mean(pred3))
print("Accuracy:", acc3)
lr3.coef_

In [ ]:
cm3 = confusion_matrix(y3_test, pred3, labels=lr3.classes_)
disp3 = ConfusionMatrixDisplay(confusion_matrix=cm3,
                              display_labels=lr3.classes_)
disp3.plot()
plt.show()

In [ ]:
print((29+22)/(29+22+17+15))

In [ ]:
lr4 = LogisticRegression()
X4_train = final_df_train[["Team_A_rating"]]
y4_train = final_df_train.Team_A_win.to_numpy().reshape(-1,1)

X4_test = final_df_test[["Team_A_rating"]]
y4_test = final_df_test.Team_A_win.to_numpy().reshape(-1,1)

In [ ]:
lr4.fit(X4_train, y4_train)
pred4 = lr4.predict(X4_test)
acc4 = accuracy_score(y4_test, pred4)
print("mean of true values:", np.mean(y4_test))
print("mean of prediction:", np.mean(pred4))
print("Accuracy:", acc4)
lr4.coef_

In [ ]:
cm4 = confusion_matrix(y4_test, pred4, labels=lr4.classes_)
disp4 = ConfusionMatrixDisplay(confusion_matrix=cm4,
                              display_labels=lr4.classes_)
disp4.plot()
plt.show()